In [1]:
import pandas as pd
import numpy as np
import json

train_identification = pd.read_csv('dm2020-hw2-nthu/data_identification.csv')
train_emotion = pd.read_csv('dm2020-hw2-nthu/emotion.csv')
sample_submit = pd.read_csv('dm2020-hw2-nthu/sampleSubmission.csv')


In [2]:
json_feature = pd.read_json('dm2020-hw2-nthu/tweets_DM.json', lines=True)

In [3]:
# 1867535 rows × 2 columns
train_identification.shape

(1867535, 2)

In [4]:
# 1455563 rows × 2 columns
train_emotion.shape

(1455563, 2)

In [5]:
#X_train = X_train.sort_values(by=['tweet_id'])
#train_emotion = train_emotion.sort_values(by=['tweet_id'])


In [6]:
data = pd.merge(train_identification, train_emotion, on=["tweet_id"])
data.head()

,tweet_id,identification,emotion
0,0x29e452,train,joy
1,0x2b3819,train,joy
2,0x2a2acc,train,trust
3,0x2a8830,train,joy
4,0x20b21d,train,anticipation


In [7]:
identification_train = data['identification'] == 'train'
identification_test = train_identification['identification'] == 'test'

train_X = data.loc[identification_train]
test_X = train_identification.loc[identification_test]

In [8]:
train_X.columns

Index(['tweet_id', 'identification', 'emotion'], dtype='object')

In [9]:
print(train_X.shape, test_X.shape)

(1455563, 3) (411972, 2)


In [10]:
test_X['emotion'] = np.nan
X = pd.concat([train_X, test_X])
X.shape

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


(1867535, 3)

In [11]:
json_feature.head()

,_crawldate,_index,_score,_source,_type
0,2015-05-23 11:42:47,hashtag_tweets,391,"{'tweet': {'hashtags': ['Snapchat'], 'tweet_id...",tweets
1,2016-01-28 04:52:09,hashtag_tweets,433,"{'tweet': {'hashtags': ['freepress', 'TrumpLeg...",tweets
2,2017-12-25 04:39:20,hashtag_tweets,232,"{'tweet': {'hashtags': ['bibleverse'], 'tweet_...",tweets
3,2016-01-24 23:53:05,hashtag_tweets,376,"{'tweet': {'hashtags': [], 'tweet_id': '0x1cd5...",tweets
4,2016-01-08 17:18:59,hashtag_tweets,989,"{'tweet': {'hashtags': [], 'tweet_id': '0x2de2...",tweets


In [15]:
json_feature['hashtags'] = np.nan; json_feature['text'] = np.nan

In [13]:
json_extend = pd.io.json.json_normalize(json_feature._source)
json_extend.head()

,tweet.hashtags,tweet.text,tweet.tweet_id
0,[Snapchat],"People who post ""add me on #Snapchat"" must be ...",0x376b20
1,"[freepress, TrumpLegacy, CNN]","@brianklaas As we see, Trump is dangerous to #...",0x2d5350
2,[bibleverse],"Confident of your obedience, I write to you, k...",0x28b412
3,[],Now ISSA is stalking Tasha 😂😂😂 <LH>,0x1cd5b0
4,[],"""Trust is not the same as faith. A friend is s...",0x2de201


In [16]:
json_feature['hashtags'] = json_extend['tweet.hashtags']
json_feature['text'] = json_extend['tweet.text']
json_feature['tweet_id'] = json_extend['tweet.tweet_id']

In [17]:
X.shape

(1867535, 3)

In [18]:
json_feature.head()

,_crawldate,_index,_score,_source,_type,hashtags,text,tweet_id
0,2015-05-23 11:42:47,hashtag_tweets,391,"{'tweet': {'hashtags': ['Snapchat'], 'tweet_id...",tweets,[Snapchat],"People who post ""add me on #Snapchat"" must be ...",0x376b20
1,2016-01-28 04:52:09,hashtag_tweets,433,"{'tweet': {'hashtags': ['freepress', 'TrumpLeg...",tweets,"[freepress, TrumpLegacy, CNN]","@brianklaas As we see, Trump is dangerous to #...",0x2d5350
2,2017-12-25 04:39:20,hashtag_tweets,232,"{'tweet': {'hashtags': ['bibleverse'], 'tweet_...",tweets,[bibleverse],"Confident of your obedience, I write to you, k...",0x28b412
3,2016-01-24 23:53:05,hashtag_tweets,376,"{'tweet': {'hashtags': [], 'tweet_id': '0x1cd5...",tweets,[],Now ISSA is stalking Tasha 😂😂😂 <LH>,0x1cd5b0
4,2016-01-08 17:18:59,hashtag_tweets,989,"{'tweet': {'hashtags': [], 'tweet_id': '0x2de2...",tweets,[],"""Trust is not the same as faith. A friend is s...",0x2de201


In [19]:
X = pd.merge(X, json_feature, on=["tweet_id"])
X.shape

(1867535, 10)

In [20]:
X.to_csv('X.csv')